In [19]:
import pandas as pd
import numpy as np
from scipy import spatial
from gensim.models import word2vec

vector_size = 250

In [20]:
df = pd.read_csv('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/csv/review_wakati.csv')

In [21]:
df.head()

,Unnamed: 0,store_id,store_name,score,station,review,wakati_review
0,0,1,中華そば しば田,4.06,仙川,開店前から長蛇の列( ロ)゚ ゚開店前10前に到着しましたが、ありつけるまで、1間半を要し...,開店 前 長蛇 列 ロ ゚ ゚ 開店 前 前 到着 間 半 そこ 東京 やはり 規模 の 夜...
1,1,2,Homemade Ramen 麦苗,4.02,大森海岸,わがままボディーの後輩との東京遠征2目の昼。実に4ぶりとなるこちらへ。当から行列のできるラー...,わがまま ボディー 後輩 東京 遠征 目 昼 実に ぶり こちら 当 行列 ラーメン屋 食べ...
2,2,3,手打式超多加水麺 ののくら,3.98,亀有,2020128 初訪TRY本で新人賞を獲った頃から行きたかった「ののくら」。今の午前中の打ち...,訪 TRY 本 新人賞 頃 のの くら 今 午前中 打ち合わせ 新小岩 午後 予定 く間 余...
3,3,4,中華そば 多賀野,3.97,荏原中延,中華そば750、大盛120動物系乾物系背脂醤油スープ中細ストレート麺ちょいカタメ豚バラチャー...,中華そば 大盛 動物 系 乾物 系 背脂 醤油 スープ 中 細 ストレート麺 ちょい カタメ...
4,4,5,麺庵ちとせ,3.97,若松河田,百名店ラーメンTOKYO2019の51店目は曙橋の『麺庵ちとせ』さんへ地下鉄の駅からお店へと...,百名 店 ラーメン TOKYO 店 目 曙 橋 麺 庵 ちとせ さん 地下鉄 駅 店 坂道 ...


In [22]:
df = df.drop(['store_id'],axis=1)

In [23]:
df = df.rename(columns={'Unnamed: 0':'store_id'}).set_index('store_id')
df

,store_name,score,station,review,wakati_review
store_id,,,,,
0,中華そば しば田,4.06,仙川,開店前から長蛇の列( ロ)゚ ゚開店前10前に到着しましたが、ありつけるまで、1間半を要し...,開店 前 長蛇 列 ロ ゚ ゚ 開店 前 前 到着 間 半 そこ 東京 やはり 規模 の 夜...
1,Homemade Ramen 麦苗,4.02,大森海岸,わがままボディーの後輩との東京遠征2目の昼。実に4ぶりとなるこちらへ。当から行列のできるラー...,わがまま ボディー 後輩 東京 遠征 目 昼 実に ぶり こちら 当 行列 ラーメン屋 食べ...
2,手打式超多加水麺 ののくら,3.98,亀有,2020128 初訪TRY本で新人賞を獲った頃から行きたかった「ののくら」。今の午前中の打ち...,訪 TRY 本 新人賞 頃 のの くら 今 午前中 打ち合わせ 新小岩 午後 予定 く間 余...
3,中華そば 多賀野,3.97,荏原中延,中華そば750、大盛120動物系乾物系背脂醤油スープ中細ストレート麺ちょいカタメ豚バラチャー...,中華そば 大盛 動物 系 乾物 系 背脂 醤油 スープ 中 細 ストレート麺 ちょい カタメ...
4,麺庵ちとせ,3.97,若松河田,百名店ラーメンTOKYO2019の51店目は曙橋の『麺庵ちとせ』さんへ地下鉄の駅からお店へと...,百名 店 ラーメン TOKYO 店 目 曙 橋 麺 庵 ちとせ さん 地下鉄 駅 店 坂道 ...
...,...,...,...,...,...
1094,旨辛ラーメン 表裏,3.50,市ケ谷,市ヶ谷ランチシリーズ(笑)まぁいつも通りと言うか麺屋さんから攻めていくわけですね〜こちらのお...,市ヶ谷 ランチ シリーズ 笑 いつも 通り 麺 屋 さん わけ こちら 店 ん カップラーメ...
1095,麺屋 ほたる,3.50,板橋,板橋駅の近くにお店があります。このは昼に煮干しラーメンを食べたので繋げて魚介系のつけ麺を食べ...,板橋駅 近く 店 このは 昼 ラーメン 魚介 系 つけ麺 こと 板橋駅 徒歩 到着 裏通り ...
1096,中華そば ばんや,3.50,下高井戸,東京都世田谷区松原。無化調、麺スープチャーシュー水等全てを厳選、こだわり抜いた中華そば店です...,東京都世田谷区松原 無化調 麺 スープ チャーシュー 水 等 全て 厳選 中華そば 店 最寄...


In [24]:
model = word2vec.Word2Vec.load('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/models/skip_w2v.model')

In [25]:
def avg_feature_vectors(words, model):
    feature_vec = np.zeros(vector_size, dtype='float32')
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError:
            pass

    return feature_vec

In [26]:
def sentence_similarity(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)

In [27]:
l = np.zeros([len(df), 250])
for i, row in df.iterrows():
    words = row.wakati_review.split(' ')
    vectors = avg_feature_vectors(words,model)
    l[i] = vectors

/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [28]:
result = {}
keyword = avg_feature_vectors(['行列', '醬油', 'ラーメン'], model)
for i, row in df.iterrows():
    score = sentence_similarity(keyword, l[i])
    result[i] = score * row.score


/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [29]:
score_sorted = sorted(result.items(), key=lambda x:x[1], reverse=True)

In [30]:
for t in score_sorted[:10]:
    row = df[df.index == t[0]]
    name, score, station = row.store_name.values[0], row.score.values[0], row.station.values[0]
    print('店名:{}, 食べログスコア:{}, 独自スコア:{:.2f}, 最寄駅:{}'.format(name,score,t[1]*100,station))
    

店名:SOBA HOUSE 金色不如帰, 食べログスコア:3.89, 独自スコア:279.62, 最寄駅:新宿御苑前
店名:中華そば しば田, 食べログスコア:4.06, 独自スコア:279.01, 最寄駅:仙川
店名:中華そば 多賀野, 食べログスコア:3.97, 独自スコア:278.87, 最寄駅:荏原中延
店名:Homemade Ramen 麦苗, 食べログスコア:4.02, 独自スコア:274.58, 最寄駅:大森海岸
店名:らぁ麺 すぎ本, 食べログスコア:3.95, 独自スコア:273.85, 最寄駅:鷺ノ宮
店名:手打式超多加水麺 ののくら, 食べログスコア:3.98, 独自スコア:273.69, 最寄駅:亀有
店名:自家製中華そば としおか, 食べログスコア:3.87, 独自スコア:272.54, 最寄駅:早稲田駅（メトロ）
店名:麺尊 RAGE, 食べログスコア:3.96, 独自スコア:271.39, 最寄駅:西荻窪
店名:鶏こく中華 すず喜, 食べログスコア:3.91, 独自スコア:269.98, 最寄駅:三鷹
店名:らーめん飛粋, 食べログスコア:3.91, 独自スコア:269.43, 最寄駅:蒲田
